### DECA Model with Facialis Parese

1. Follow README_Install_Mattis.md

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

Raw testing

In [ ]:
!CUDA_HOME=$CONDA_PREFIX python demos/demo_reconstruct.py -i TestSamples/examples --saveDepth True --saveObj True --rasterizer_type pytorch3d

image = mpimg.imread("TestSamples/examples/results/alfw1_vis.jpg")
plt.imshow(image)
plt.axis('off')
plt.show()

Raw Testing - facialis parese patient


## Facialis Parese FLAME model with disassembled sides

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

0. Loading Image

In [ ]:
import os
input_path = "TestSamples/Brenken_Rudolf/"

input_image = "2017-04-12_007.png"

# Extrahiere den Dateinamen mit der Erweiterung
filename_with_extension = os.path.basename(input_image)

# Entferne die Dateierweiterung
filename_without_extension, extension = os.path.splitext(filename_with_extension)

# Teile die Zeichenkette am Unterstrich, um das gewünschte Datum und die Nummer zu erhalten
date_part, number_part = filename_without_extension.split("_")

print("Datum:", date_part)
print("Nummer:", number_part)

image = cv2.imread(input_path+input_image)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#==========Plot==========
plt.imshow(image)
plt.axis('off')
plt.show()

1. Decoupling images to left and right side

In [ ]:
import mediapipe as mp
import cv2

#Using Mediapipe to decouple the image to left and right side via the midpoints of the eyes to make it more "middle" than just using the image shape

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()

image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype('uint8')

results = face_mesh.process(image_rgb)
landmarks = results.multi_face_landmarks[0].landmark

left_eye_inner = np.array([landmarks[159].x, landmarks[159].y, landmarks[159].z])
right_eye_inner = np.array([landmarks[386].x, landmarks[386].y, landmarks[386].z])

eye_midpoint = (left_eye_inner + right_eye_inner) / 2

eye_midpoint_pixel = (int(eye_midpoint[0] * image.shape[1]), int(eye_midpoint[1] * image.shape[0]))

print(eye_midpoint_pixel)

right_half = image[:, :eye_midpoint_pixel[0], :]
left_half = image[:, eye_midpoint_pixel[0]:, :]

#==========Plot==========
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.imshow(image)
plt.title('Original')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(left_half)
plt.title('Left Half')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(right_half)
plt.title('Right half')
plt.axis('off')

plt.show()

2. Flipping and adjusting the images

In [ ]:
left_half_flipped = np.flip(left_half, axis=1)
right_half_flipped = np.flip(right_half, axis=1)

left_half_full_image = np.concatenate([left_half_flipped, left_half], axis=1)
right_half_full_image = np.concatenate([right_half, right_half_flipped], axis=1)

left_image_path = input_path + "left_half_full_image.png"
right_image_path = input_path + "right_half_full_image.png"

plt.imsave(left_image_path, left_half_full_image)
plt.imsave(right_image_path, right_half_full_image)

#==========Plot==========
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.imshow(image)
plt.title('Original')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(left_half_full_image)
plt.title('Left Half mirrored')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(right_half_full_image)
plt.title('Right half mirrored')
plt.axis('off')

plt.show()

3. Adjusting FLAME to both sides

In [ ]:
output_path = "TestSamples/Brenken_Rudolf/results/leftright"

In [ ]:
!CUDA_HOME=$CONDA_PREFIX

In [ ]:
#--iscrop False , because else the image would be cropped and the face would not be full
%run demos/demo_reconstruct.py -i $left_image_path --saveDepth True --savefolder $output_path --saveObj True --rasterizer_type pytorch3d --render_orig True --iscrop False

image = mpimg.imread(output_path + "/left_half_full_image_vis_original_size.jpg")
plt.imshow(image)
plt.axis('off')
plt.show()

In [ ]:
%run demos/demo_reconstruct.py -i $right_image_path --saveDepth True --savefolder $output_path --saveObj True --rasterizer_type pytorch3d --render_orig True

image = mpimg.imread(output_path + "/right_half_full_image_vis_original_size.jpg")
plt.imshow(image)
plt.axis('off')
plt.show()

4. Bringing both sides together and building a coherent model

In [ ]:
def blending_sides(vertices_left, vertices_right, link:str):

    vertice_exp_lr = np.zeros((5023, 3))
    template_vertices = vertices_right
    vertice_max = template_vertices[:,0].max()
    vertice_min = template_vertices[:,0].min()
    vertice_diff = vertice_max - vertice_min
    print(vertice_diff)

    # Iterate through each vertex and apply the specified blending method
    for i in range(0, 5023):
        rel_dist = (template_vertices[i, 0] - vertice_min) / vertice_diff
        if link == "linear":
            vertice_exp_lr[i] = rel_dist * vertices_right[i] + (1 - rel_dist) * vertices_left[i]
        elif link == "exponential":
            rel_dist_scaled = rel_dist * 2 - 1  # Skaliere von 0-1 zu -1-1, um symmetrische Exponentialfunktion zu erhalten
            rel_dist_transformed = (np.exp(rel_dist_scaled) - 1) / (np.exp(1) - 1)  # Hier exponentielle Transformation von -1-1 auf 0-1
            vertice_exp_lr[i] = rel_dist_transformed * vertices_right[i] + (1 - rel_dist_transformed) * vertices_left[i]
        elif link == "binary":
            vertice_exp_lr[i] = vertices_right[i]+0.1*vertices_left[i] if rel_dist > 0.5 else vertices_left[i]
            
    return vertice_exp_lr

In [ ]:
import pyrender
import trimesh
def viewer(vertices, faces):
    """Interactive visualization via Pyrender Viewer of a (face) mesh

    Args:
        vertices (np.ndarray): 3D vertices of the mesh.
        faces (np.ndarray): Faces defining the mesh.

    Returns:
        None
    """
    vertex_colors = np.ones([vertices.shape[0], 4]) * [1.0, 1.0, 1.0, 1.0]

    tri_mesh = trimesh.Trimesh(vertices, faces, vertex_colors=vertex_colors)
    tri_mesh.export('output_mesh.obj')
    mesh = pyrender.Mesh.from_trimesh(tri_mesh)
    scene = pyrender.Scene()
    scene.add(mesh)
    pyrender.Viewer(scene, use_raymond_lighting=True)


In [ ]:
vertices_left = np.loadtxt(output_path+"/left_half_full_image/left_half_full_image_vertice.txt")
vertices_right = np.loadtxt(output_path+"/right_half_full_image/right_half_full_image_vertice.txt")
faces = np.loadtxt(output_path+"/left_half_full_image/left_half_full_image_faces.txt")

#Problem exponential und linear: Änderung kaum zu erkennen
vertice_lr = blending_sides(vertices_left, vertices_right, link="binary")

vertex_colors = np.ones([vertice_lr.shape[0], 4]) * [1.0, 1.0, 1.0, 1.0]
tri_mesh = trimesh.Trimesh(vertice_lr, faces, vertex_colors=vertex_colors)
viewer(vertice_lr, faces)

Building a shape and pose model and only using the expression parameters for the sides 

1. Getting a template Model without specified parameters

2. Realized: Wen need to get the expression parameters and take them to FLAME_PyTorch, so the other way around; it's easier

3. Getting the expression parameters

Left

In [ ]:
from decalib.deca import DECA
from decalib.datasets import datasets 
from decalib.utils import util
from decalib.utils.config import cfg as deca_cfg
from skimage.io import imread
import torch
from tqdm import tqdm

# Beispielaufruf
image_path = left_image_path
device = 'cuda'
deca = DECA(config = deca_cfg, device='cuda')

testdata = datasets.TestData(image_path)
deca = DECA(config = deca_cfg, device=device)
# for i in range(len(testdata)):
for i in tqdm(range(len(testdata))):
    name = testdata[i]['imagename']
    images = testdata[i]['image'].to(device)[None,...]
    with torch.no_grad():
        codedict = deca.encode(images)

# Führen Sie das DECA-Modell aus, um die Expressionsparameter zu erhalten
expressions_params = codedict['exp']

# Drucken Sie die Expressionsparameter
print("Expressionsparameter:")
print(expressions_params)

torch.save(expressions_params, 'expressions_params_left.pth')

Right

In [ ]:
from decalib.deca import DECA
from decalib.datasets import datasets 
from decalib.utils import util
from decalib.utils.config import cfg as deca_cfg
from skimage.io import imread
import torch
from tqdm import tqdm

# Beispielaufruf
image_path = right_image_path
device = 'cuda'
deca = DECA(config = deca_cfg, device='cuda')

testdata = datasets.TestData(image_path)
deca = DECA(config = deca_cfg, device=device)
# for i in range(len(testdata)):
for i in tqdm(range(len(testdata))):
    name = testdata[i]['imagename']
    images = testdata[i]['image'].to(device)[None,...]
    with torch.no_grad():
        codedict = deca.encode(images)

# Führen Sie das DECA-Modell aus, um die Expressionsparameter zu erhalten
expressions_params = codedict['exp']

# Drucken Sie die Expressionsparameter
print("Expressionsparameter:")
print(expressions_params)

torch.save(expressions_params, 'expressions_params_right.pth')

In [ ]:
import DePa

!CUDA_HOME=$CONDA_PREFIX

image_path = "TestSamples/Brenken_Rudolf/2017-10-24_00-00_Brenken-Rudolf_06/2017-10-24_001.jpg"
results_path = "TestResults/testing/"

DePa.decouple(image_path, results_path)


In [2]:
from decalib.models.encoders import ResnetEncoder
import numpy as np
from decalib.datasets import datasets 
import torch
import os
from decalib.utils import util

def get_shape_params(image_path: str, device='cuda'):
    """Function for getting the shape paramters of an image
    """
    
    testdata__ = datasets.TestData(image_path)
    for i in range(len(image_path)):
        imgs_batch = testdata__[1]['image'].to('cuda')[None,...]
        codedict = encode(imgs_batch)
            
    return codedict['shape']

n_shape = 100
n_tex = 50
n_exp = 50
n_pose = 6
n_cam = 3
n_light = 27

model_path = "/home/dietrich/Testing/DECA/DECA/data/deca_model.tar"
num_params = n_shape+n_tex+n_exp+n_pose+n_cam+n_light
E_flame  = ResnetEncoder(outsize=num_params).to('cuda')
param_dict = {'shape': n_shape, 'tex': n_tex, 'exp': n_exp, 'pose': n_pose, 'cam': n_cam, 'light': n_light}
checkpoint = torch.load(model_path)
util.copy_state_dict(E_flame.state_dict(), checkpoint['E_flame'])
E_flame.eval()

def encode(imgs_batch):
        with torch.no_grad():
                parameters = E_flame(imgs_batch)
        code_dict__ = decompose_code(parameters, param_dict)
        code_dict__['images'] = imgs_batch
        return code_dict__

def decompose_code(concatCode, num_dict__):
        ''' Convert a flattened parameter vector to a dictionary of parameters
        code_dict.keys() = [('shape',) 'tex', 'exp', 'pose', 'cam', 'light']
        '''
        code_dict_ = {}
        start_ind_ = 0
        for key in num_dict__:
            end_ind___      = start_ind_+int(num_dict__[key])
            code_dict_[key] = concatCode[:, start_ind_:end_ind___]
            start_ind_      = end_ind___
            if key == 'light':
                code_dict_[key] = code_dict_[key].reshape(code_dict_[key].shape[0], 9, 3)
        return code_dict_

In [ ]:
from decalib.deca import DECA
from decalib.datasets import datasets 
from decalib.utils import util
from decalib.utils.config import cfg as deca_cfg
from skimage.io import imread
import torch
from tqdm import tqdm

image_path = "/home/dietrich/Testing/DECA/DECA/TestSamples/Brenken_Rudolf/2017-10-24_00-00_Brenken-Rudolf_06/2017-10-24_001.jpg"
# Initialization
deca = DECA()
device = 'cuda'
#Get the data in the right format
test_img = datasets.TestData(image_path)
#Get the codedictionarys
for i in tqdm(range(len(test_img))):
    name = test_img[i]['imagename']
    images_l = test_img[i]['image'].to(device)[None,...]
    with torch.no_grad(): 
        codedict = deca.encode(images_l)
        shape_params = get_shape_params(image_path)
        opdict, visdict = deca.decode(codedict, train_bool=True, shape_params=shape_params) #tensor
        deca.save_obj(name + '_without_shape.obj', opdict)
        print(codedict['exp'])
